In [1]:
import torch
import torch.autograd as autograd         # computation graph
from torch import Tensor                  # tensor node in the computation graph
import torch.nn as nn                     # neural networks
import torch.optim as optim               # optimizers e.g. gradient descent, ADAM, etc.

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.ticker
from torch.nn.parameter import Parameter
import matplotlib as mpl

import numpy as np
import time
from pyDOE import lhs         #Latin Hypercube Sampling
import scipy.io
from scipy.io import savemat

from smt.sampling_methods import LHS

#Set default dtype to float32
torch.set_default_dtype(torch.float)

#PyTorch random number generator
torch.manual_seed(1234)

# Random number generators in other libraries
np.random.seed(1234)

# Device configuration
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')

print(device)

if device == 'cuda': 
    print(torch.cuda.get_device_name())

    

cuda:1


In [2]:
lr_tune = np.array([0.05,0.1,0.25,0.5,1])

In [3]:
# def true_2D_1(xt): #True function for 2D_1 Klein Gordon Equation x \in [-50,50] , t \in [0,10]
#     y = xt[:,0]*np.cos(xt[:,1])
#     return y.reshape(-1,1)    

In [4]:
loss_thresh = 0.5


x = np.linspace(0,1,500).reshape(-1,1)
y = np.linspace(0,1,500).reshape(-1,1)

X,Y = np.meshgrid(x,y)

X = X.flatten('F').reshape(-1,1)
Y = Y.flatten('F').reshape(-1,1)
  
xy = np.hstack((X,Y))

# bound_pts_1 = (X == 0).reshape(-1,)
# bound_pts_2 = np.logical_and(Y == 0,X != 0).reshape(-1,)
# bound_pts_3 = np.logical_and(X == 1,Y != 0).reshape(-1,) 
# bound_pts_4 = np.logical_and(Y == 1,X != 1).reshape(-1,) 

# xy_bound_1 = xy[bound_pts_1,:]
# xy_bound_2 = xy[bound_pts_2,:]
# xy_bound_3 = xy[bound_pts_3,:]
# xy_bound_4 = xy[bound_pts_4,:]

# u_bound_1 = 1000*np.ones((np.shape(xy_bound_1)[0],1))
# u_bound_2 = 800*np.ones((np.shape(xy_bound_2)[0],1))
# u_bound_3 = 500*np.ones((np.shape(xy_bound_3)[0],1))
# u_bound_4 = np.zeros((np.shape(xy_bound_4)[0],1))

# xy_bound = np.vstack((xy_bound_1,xy_bound_2,xy_bound_3,xy_bound_4))
# u_bound = np.vstack((u_bound_1,u_bound_2,u_bound_3,u_bound_4))

xy_test_tensor = torch.from_numpy(xy).float().to(device)

lb_xy = xy[0]
ub_xy = xy[-1]

In [5]:
fea_data = scipy.io.loadmat('./../ES_FEA.mat')

xy = np.array(fea_data['xy'])
u_true = np.array(fea_data['u'])

xy_test_tensor = torch.from_numpy(xy).float().to(device)
u_true_norm = np.linalg.norm(u_true,2)

In [6]:
def trainingdata(N_T,N_f,seed):
    '''Boundary Conditions''' 
    
    np.random.seed(seed)
    N_t = int(N_T/4)
    
    x_BC1 = np.random.uniform(size = N_t).reshape(-1,1)
    y_BC1 = np.zeros((N_t,1))
    u_BC1 = 0*np.ones((N_t,1))
    
    x_BC2 = np.ones((N_t,1))
    y_BC2 = np.random.uniform(size = N_t).reshape(-1,1) 
    u_BC2 = 1000*np.ones((N_t,1))
    
    x_BC3 = np.random.uniform(size = N_t).reshape(-1,1)
    y_BC3 = np.ones((N_t,1)) 
    u_BC3 = 0*np.ones((N_t,1))
    
    x_BC4 = np.zeros((N_t,1))
    y_BC4 = np.random.uniform(size = N_t).reshape(-1,1) 
    u_BC4 = 1000*np.ones((N_t,1))
    
    XY_1 = np.hstack((x_BC1,y_BC1))
    XY_2 = np.hstack((x_BC2,y_BC2))
    XY_3 = np.hstack((x_BC3,y_BC3))
    XY_4 = np.hstack((x_BC4,y_BC4))
    
    xy_BC = np.vstack((XY_1,XY_2,XY_3,XY_4)) #choose indices from  set 'idx' (x,t)
    u_BC = np.vstack((u_BC1,u_BC2,u_BC3,u_BC4))
    
    '''Collocation Points'''

    # Latin Hypercube sampling for collocation points 
    # N_f sets of tuples(x,t)
    x01 = np.array([[0.0,1.0],[0.0,1.0]])
    sampling = LHS(xlimits=x01,random_state =seed)
    samples = sampling(N_f)
    
    xy_coll = lb_xy + (ub_xy - lb_xy)*samples
    
    xy_coll = np.vstack((xy_coll, xy_BC)) # append training points to collocation points 

    return xy_coll, xy_BC, u_BC

In [7]:
class Sequentialmodel(nn.Module):
    
    def __init__(self,layers):
        super().__init__() #call __init__ from parent class 
              
        'activation function'
        self.activation = nn.Sigmoid()

     
        'loss function'
        self.loss_function = nn.MSELoss(reduction ='mean')
        
        'Initialise neural network as a list using nn.Modulelist'  
        self.linears = nn.ModuleList([nn.Linear(layers[i], layers[i+1]) for i in range(len(layers)-1)])
        self.iter = 0
        
        # std = gain * sqrt(2/(input_dim+output_dim))
        for i in range(len(layers)-1):
            nn.init.xavier_normal_(self.linears[i].weight.data, gain=1.0)
            # set biases to zero
            nn.init.zeros_(self.linears[i].bias.data)   
        
        self.beta = Parameter(torch.ones((50,len(layers)-2)))
        self.beta.requiresGrad = True
        
            
    'foward pass'
    def forward(self,xy):
        if torch.is_tensor(xy) != True:         
            xy = torch.from_numpy(xy)                
        
        ubxy = torch.from_numpy(ub_xy).float().to(device)
        lbxy = torch.from_numpy(lb_xy).float().to(device)
    
                      
        #preprocessing input 
        xy = (xy - lbxy)/(ubxy - lbxy)
        
        #convert to float
        a = xy.float()
        
        for i in range(len(layers)-2):
            z = self.linears[i](a)
            a = z*self.activation(self.beta[:,i]*z)
            
        a = self.linears[-1](a) 
         
        return a
                        
    def loss_BC(self,xy,u):
                
        loss_bc = self.loss_function(self.forward(xy), u)
                
        return loss_bc
    
    def loss_PDE(self, xy_coll, f_hat):
        
        g = xy_coll.clone()             
        g.requires_grad = True
        u = self.forward(g) 
        
        u_x_y = autograd.grad(u,g,torch.ones([xy_coll.shape[0], 1]).to(device), retain_graph=True, create_graph=True,allow_unused = True)[0]
        
        u_xx_yy = autograd.grad(u_x_y,g,torch.ones(xy_coll.shape).to(device), create_graph=True,allow_unused = True)[0]

        #du_dx = u_x_t[:,[0]]
        
        d2u_dx2 = u_xx_yy[:,[0]]
        d2u_dy2 = u_xx_yy[:,[1]]    
        

        f = d2u_dx2 + d2u_dy2
        
        loss_f = self.loss_function(f,f_hat)
                
        return loss_f
    
    def loss(self,xy_BC,u_BC,xy_coll,f_hat):

        loss_BC = self.loss_BC(xy_BC,u_BC)
        loss_f = self.loss_PDE(xy_coll,f_hat)
        
        loss_val = loss_BC + loss_f
        
        return loss_val
     
    'callable for optimizer'  
    def closure(self):
        
        optimizer.zero_grad()
        loss = self.loss(xy_BC, u_BC, xy_coll,f_hat)
        self.train_loss.append(loss.cpu().detach().numpy())
        
        u_pred = self.test(xy_test_tensor)
        #self.test_loss.append(np.mean(np.square(u_pred.reshape(-1,1) - u_true.reshape(-1,1)))) #Commented because no true values yet
        self.beta_val.append(self.beta.cpu().detach().numpy())
        
        #print(self.iter,"Train Loss",self.train_loss[-1],"Test Loss",self.test_loss[-1])
        print(self.iter,"Train Loss",self.train_loss[-1])
        
        loss.backward()
        
        self.iter += 1
  

        return loss        
    'test neural network'
    def test(self):
        u_pred = self.forward(xy_test_tensor)
        u_pred = u_pred.cpu().detach().numpy()
   
        return u_pred

    def test_loss(self):
        u_pred = self.test()
        
        test_mse = np.mean(np.square(u_pred.reshape(-1,1) - u_true.reshape(-1,1)))
        test_re = np.linalg.norm(u_pred.reshape(-1,1) - u_true.reshape(-1,1),2)/u_true_norm
     
        
        return test_mse, test_re 

In [8]:
def data_update(loss_np):
    train_loss.append(loss_np)
    beta_val.append(PINN.beta.cpu().detach().numpy())
    
    test_mse, test_re = PINN.test_loss()
    test_mse_loss.append(test_mse)
    test_re_loss.append(test_re)

In [9]:
def train_step(xy_BC,u_BC,xy_coll,f_hat,seed):

    def closure():
        optimizer.zero_grad()
        loss = PINN.loss(xy_BC,u_BC,xy_coll,f_hat)
        loss.backward()
        #print(loss.cpu().detach().numpy())
        return loss

    optimizer.step(closure)

In [10]:
def train_model(max_iter,rep): 
    print(rep) 
    torch.manual_seed(rep*9)
    start_time = time.time() 
    thresh_flag = 0

    xy_coll_np_array, xy_BC_np_array, u_BC_np_array = trainingdata(N_T,N_f,rep*22)
        
    xy_coll = torch.from_numpy(xy_coll_np_array).float().to(device)
    xy_BC = torch.from_numpy(xy_BC_np_array).float().to(device)
    u_BC = torch.from_numpy(u_BC_np_array).float().to(device)
        
    f_hat = torch.zeros(xy_coll.shape[0],1).to(device)
    
    nan_flag = 0
    for i in range(max_iter):
        train_step(xy_BC,u_BC,xy_coll,f_hat,i)
        loss_np = PINN.loss(xy_BC,u_BC,xy_coll,f_hat).cpu().detach().numpy()
        
        if(thresh_flag == 0):
            if(loss_np < loss_thresh):
                time_threshold[rep] = time.time() - start_time
                epoch_threshold[rep] = i+1          
                thresh_flag = 1       
        data_update(loss_np)
        
        print(i,"Train Loss",train_loss[-1],"Test MSE",test_mse_loss[-1],"Test RE",test_re_loss[-1])   
        
        if(np.isnan(loss_np)):
            nan_flag =1
            print("NAN BREAK!")
            break
            
    elapsed_time[rep] = time.time() - start_time  
    print('Training time: %.2f' % (elapsed_time[rep]))
    
    return nan_flag

In [11]:
nan_tune = []
for tune_reps in range(20):      
    
    max_reps = 10
    max_iter = 100 #200

    label = "MW_swish_tune" + str(tune_reps)

    train_loss_full = []
    test_mse_full = []
    test_re_full = []
    beta_full = []
    elapsed_time= np.zeros((max_reps,1))
    time_threshold = np.empty((max_reps,1))
    time_threshold[:] = np.nan
    epoch_threshold = max_iter*np.ones((max_reps,1))


    N_T = 5000 #Total number of data points for 'y'
    N_f = 10000 #Total number of collocation points 


    for reps in range(max_reps):
        print(label)
        print(reps)
        train_loss = []
        test_mse_loss = []
        test_re_loss = []
        beta_val = []

        torch.manual_seed(reps*36)

        layers = np.array([2,50,50,50,50,50,50,50,50,50,1]) #9 hidden layers

        PINN = Sequentialmodel(layers)

        PINN.to(device)

        'Neural Network Summary'
        print(PINN)

        params = list(PINN.parameters())

        optimizer = torch.optim.LBFGS(PINN.parameters(), lr=lr_tune[tune_reps], 
                                  max_iter = 20, 
                                  max_eval = 30, 
                                  tolerance_grad = 1e-08, 
                                  tolerance_change = 1e-08, 
                                  history_size = 100, 
                                  line_search_fn = 'strong_wolfe')





        nan_flag = train_model(max_iter,reps)


        torch.save(PINN.state_dict(),label+'_'+str(reps)+'.pt')
        train_loss_full.append(train_loss)
        test_mse_full.append(test_mse_loss)
        test_re_full.append(test_re_loss)
        #elapsed_time[reps] = time.time() - start_time
        beta_full.append(beta_val)
        
        if(nan_flag == 1):
            nan_tune.append(tune_reps)
            break


        print('Training time: %.2f' % (elapsed_time[reps]))

    mdic = {"train_loss": train_loss_full,"test_mse_loss": test_mse_full,"test_re_loss": test_re_full,"Time": elapsed_time, "beta": beta_full, "label": label,"Thresh Time": time_threshold,"Thresh epoch": epoch_threshold}
    savemat(label+'.mat', mdic)

MW_swish_tune0
0
Sequentialmodel(
  (activation): Sigmoid()
  (loss_function): MSELoss()
  (linears): ModuleList(
    (0): Linear(in_features=2, out_features=50, bias=True)
    (1): Linear(in_features=50, out_features=50, bias=True)
    (2): Linear(in_features=50, out_features=50, bias=True)
    (3): Linear(in_features=50, out_features=50, bias=True)
    (4): Linear(in_features=50, out_features=50, bias=True)
    (5): Linear(in_features=50, out_features=50, bias=True)
    (6): Linear(in_features=50, out_features=50, bias=True)
    (7): Linear(in_features=50, out_features=50, bias=True)
    (8): Linear(in_features=50, out_features=50, bias=True)
    (9): Linear(in_features=50, out_features=1, bias=True)
  )
)
0
0 Train Loss 250000.12 Test MSE 75079.02148585906 Test RE 0.47939703507920906
1 Train Loss 249999.3 Test MSE 75080.21649417964 Test RE 0.47940085026722606
2 Train Loss 249999.25 Test MSE 75080.36619002427 Test RE 0.4794013281845974
3 Train Loss 249999.25 Test MSE 75080.4068477551

96 Train Loss 249999.2 Test MSE 75080.49044514255 Test RE 0.47940172487981303
97 Train Loss 249999.2 Test MSE 75080.49044514255 Test RE 0.47940172487981303
98 Train Loss 249999.2 Test MSE 75080.49044514255 Test RE 0.47940172487981303
99 Train Loss 249999.2 Test MSE 75080.49044514255 Test RE 0.47940172487981303
Training time: 65.23
Training time: 65.23
MW_swish_tune0
1
Sequentialmodel(
  (activation): Sigmoid()
  (loss_function): MSELoss()
  (linears): ModuleList(
    (0): Linear(in_features=2, out_features=50, bias=True)
    (1): Linear(in_features=50, out_features=50, bias=True)
    (2): Linear(in_features=50, out_features=50, bias=True)
    (3): Linear(in_features=50, out_features=50, bias=True)
    (4): Linear(in_features=50, out_features=50, bias=True)
    (5): Linear(in_features=50, out_features=50, bias=True)
    (6): Linear(in_features=50, out_features=50, bias=True)
    (7): Linear(in_features=50, out_features=50, bias=True)
    (8): Linear(in_features=50, out_features=50, bias

91 Train Loss 250000.48 Test MSE 75080.75254841306 Test RE 0.47940256166610445
92 Train Loss 250000.48 Test MSE 75080.75254841306 Test RE 0.47940256166610445
93 Train Loss 250000.48 Test MSE 75080.75254841306 Test RE 0.47940256166610445
94 Train Loss 250000.48 Test MSE 75080.75254841306 Test RE 0.47940256166610445
95 Train Loss 250000.48 Test MSE 75080.75254841306 Test RE 0.47940256166610445
96 Train Loss 250000.48 Test MSE 75080.75254841306 Test RE 0.47940256166610445
97 Train Loss 250000.48 Test MSE 75080.75254841306 Test RE 0.47940256166610445
98 Train Loss 250000.48 Test MSE 75080.75254841306 Test RE 0.47940256166610445
99 Train Loss 250000.48 Test MSE 75080.75254841306 Test RE 0.47940256166610445
Training time: 61.14
Training time: 61.14
MW_swish_tune0
2
Sequentialmodel(
  (activation): Sigmoid()
  (loss_function): MSELoss()
  (linears): ModuleList(
    (0): Linear(in_features=2, out_features=50, bias=True)
    (1): Linear(in_features=50, out_features=50, bias=True)
    (2): Linea

/home/smartlab/anaconda3/lib/python3.7/site-packages/scipy/io/matlab/mio5.py:493: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  narr = np.asanyarray(source)


0 Train Loss 249996.48 Test MSE 75079.7683155964 Test RE 0.47939941941416947
1 Train Loss 249996.48 Test MSE 75079.82719455186 Test RE 0.47939960739111825
2 Train Loss 249996.48 Test MSE 75079.88620800622 Test RE 0.4793997957973943
3 Train Loss 249996.48 Test MSE 75079.9473567149 Test RE 0.47939999102060227
4 Train Loss 249996.48 Test MSE 75080.00897326454 Test RE 0.47940018773735765
5 Train Loss 249996.45 Test MSE 75080.16092451697 Test RE 0.479400672855987
6 Train Loss 249996.44 Test MSE 75080.32260863252 Test RE 0.4794011890471559
7 Train Loss 249996.33 Test MSE 75080.95009411978 Test RE 0.47940319234608786
8 Train Loss 249996.28 Test MSE 75081.14827739181 Test RE 0.4794038250607142
9 Train Loss 249116.98 Test MSE 74588.89257685725 Test RE 0.47782967929031206
10 Train Loss 235395.89 Test MSE 69379.58528919285 Test RE 0.46084182902816045
11 Train Loss 162393.9 Test MSE 48124.861618910014 Test RE 0.38381397939996514
12 Train Loss 94980.92 Test MSE 22494.103070789697 Test RE 0.26240395

...

77 Train Loss 40555.883 Test MSE 6085.009843225644 Test RE 0.13647926796124152
78 Train Loss 40555.883 Test MSE 6085.009843225644 Test RE 0.13647926796124152
79 Train Loss 40555.883 Test MSE 6085.009843225644 Test RE 0.13647926796124152
80 Train Loss 40555.883 Test MSE 6085.009843225644 Test RE 0.13647926796124152
81 Train Loss 40555.883 Test MSE 6085.009843225644 Test RE 0.13647926796124152
82 Train Loss 40555.883 Test MSE 6085.009843225644 Test RE 0.13647926796124152
83 Train Loss 40555.883 Test MSE 6085.009843225644 Test RE 0.13647926796124152
84 Train Loss 40555.883 Test MSE 6085.009843225644 Test RE 0.13647926796124152
85 Train Loss 40555.883 Test MSE 6085.009843225644 Test RE 0.13647926796124152
86 Train Loss 40555.883 Test MSE 6085.009843225644 Test RE 0.13647926796124152
87 Train Loss 40555.883 Test MSE 6085.009843225644 Test RE 0.13647926796124152
88 Train Loss 40555.883 Test MSE 6085.009843225644 Test RE 0.13647926796124152
89 Train Loss 40555.883 Test MSE 6085.009843225644 T

72 Train Loss 41440.543 Test MSE 5720.855774346118 Test RE 0.13233250738050123
73 Train Loss 41404.484 Test MSE 5712.843844151951 Test RE 0.13223981055998008
74 Train Loss 41347.668 Test MSE 5694.0643366822815 Test RE 0.1320222794298538
75 Train Loss 41322.285 Test MSE 5679.201388449451 Test RE 0.13184986107027277
76 Train Loss 41306.625 Test MSE 5669.706394386529 Test RE 0.1317395958000858
77 Train Loss 41300.297 Test MSE 5671.450699628376 Test RE 0.13175985931815962
78 Train Loss 41287.168 Test MSE 5667.70783059813 Test RE 0.13171637473943656
79 Train Loss 41271.574 Test MSE 5658.977409924455 Test RE 0.1316148890442837
80 Train Loss 41255.418 Test MSE 5645.4855899529475 Test RE 0.1314579009597493
81 Train Loss 41246.418 Test MSE 5642.352462985482 Test RE 0.13142141769809737
82 Train Loss 41227.37 Test MSE 5646.685442795797 Test RE 0.1314718697985532
83 Train Loss 41212.46 Test MSE 5655.425416925926 Test RE 0.13157357693856303
84 Train Loss 41203.906 Test MSE 5659.066627703529 Test RE

68 Train Loss 250000.14 Test MSE 75081.75609446947 Test RE 0.4794057655560911
69 Train Loss 250000.14 Test MSE 75081.75609446947 Test RE 0.4794057655560911
70 Train Loss 250000.14 Test MSE 75081.75609446947 Test RE 0.4794057655560911
71 Train Loss 250000.14 Test MSE 75081.75609446947 Test RE 0.4794057655560911
72 Train Loss 250000.14 Test MSE 75081.75609446947 Test RE 0.4794057655560911
73 Train Loss 250000.14 Test MSE 75081.75609446947 Test RE 0.4794057655560911
74 Train Loss 250000.14 Test MSE 75081.75609446947 Test RE 0.4794057655560911
75 Train Loss 250000.14 Test MSE 75081.75609446947 Test RE 0.4794057655560911
76 Train Loss 250000.14 Test MSE 75081.75609446947 Test RE 0.4794057655560911
77 Train Loss 250000.14 Test MSE 75081.75609446947 Test RE 0.4794057655560911
78 Train Loss 250000.14 Test MSE 75081.75609446947 Test RE 0.4794057655560911
79 Train Loss 250000.14 Test MSE 75081.75609446947 Test RE 0.4794057655560911
80 Train Loss 250000.14 Test MSE 75081.75609446947 Test RE 0.479

63 Train Loss 250000.08 Test MSE 75081.46503154226 Test RE 0.47940483631982966
64 Train Loss 250000.08 Test MSE 75081.46503154226 Test RE 0.47940483631982966
65 Train Loss 250000.08 Test MSE 75081.46503154226 Test RE 0.47940483631982966
66 Train Loss 250000.08 Test MSE 75081.46503154226 Test RE 0.47940483631982966
67 Train Loss 250000.08 Test MSE 75081.46503154226 Test RE 0.47940483631982966
68 Train Loss 250000.08 Test MSE 75081.46503154226 Test RE 0.47940483631982966
69 Train Loss 250000.08 Test MSE 75081.46503154226 Test RE 0.47940483631982966
70 Train Loss 250000.08 Test MSE 75081.46503154226 Test RE 0.47940483631982966
71 Train Loss 250000.08 Test MSE 75081.46503154226 Test RE 0.47940483631982966
72 Train Loss 250000.08 Test MSE 75081.46503154226 Test RE 0.47940483631982966
73 Train Loss 250000.08 Test MSE 75081.46503154226 Test RE 0.47940483631982966
74 Train Loss 250000.08 Test MSE 75081.46503154226 Test RE 0.47940483631982966
75 Train Loss 250000.08 Test MSE 75081.46503154226 T

IndexError: index 5 is out of bounds for axis 0 with size 5

In [ ]:
u_pred = PINN.test()

In [ ]:
fig, ax = plt.subplots(1,1)
cmap = plt.cm.jet

img3 = ax.imshow(np.transpose(np.flipud(np.transpose(u_pred.reshape(500,500)))),vmin = 0,vmax = 1000,cmap = cmap,extent=[0,1,0,1],aspect = 0.75)


In [13]:
import scipy.io as sio
import numpy as np

for tune_reps in range(5):
    label = "MW_swish_tune"+str(tune_reps)+".mat"
    data = sio.loadmat(label)
    re = np.array(data["test_re_loss"])
    print(tune_reps," ",np.mean(re[:,-1]))

0   [[0.47940141 0.47940322 0.47940358 0.47940418 0.47940448 0.47940477
  0.4794049  0.47940503 0.47940528 0.4794054  0.47940553 0.47940565
  0.47940577 0.47940601 0.47940627 0.47940639 0.47940652 0.4794068
  0.47940707 0.47940722 0.47940736 0.4794075  0.47940803 0.47940819
  0.47940875 0.47941014 0.47941061 0.47941084 0.47941368 0.4794159
  0.47942433 0.47939733 0.47939263 0.47939242 0.47938299        nan]]
1   0.375826297376343
2   0.3742012560984197
3   [[0.47940462 0.47940485 0.47940497 0.47940506 0.47940513 0.47940522
  0.47940534 0.4794055  0.47940572 0.47940642        nan]]
4   [[0.47940536 0.47940589 0.47940656        nan]]


In [15]:
lr_tune[2]

0.25